In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_pinecone import Pinecone as PineconeStore
import os
import pinecone
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv

/home/yvesloic/anaconda3/envs/mchatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("/home/yvesloic/Documents/informatique/Défi/text-generative-chatbot-using-llama2/data")

In [6]:
#extracted_data

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 260


In [9]:
#text_chunks

In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
#query_result

In [ ]:
#Initializing the Pinecone
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


#Creating Embeddings for Each of The Text Chunks & storing

vector_database_index = PineconeStore.from_documents(
                                            index_name = "university-chatbot", 
                                            documents = text_chunks, 
                                            embedding = embeddings)

In [17]:
#Initializing the Pinecone if you didn't execute the above cells
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

index="university-chatbot"
#If we already have an index we can load it like this
vector_database_index=PineconeStore.from_existing_index(index, embeddings)

query = "What is ISPM"

docs=vector_database_index.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='The institution E S P M  \nItaosy is situate in Itaosy and offers Sciences de la Santé/Licence with \nemphasis on -      Sage-femme \n-      Infirmier  Généraliste  \n-      Bio technicien d’a nalyses médicales and have government \napprobation referenceArrêté n°9190/2013 - MESupReS du 23 avril 2013  \nRnv : Arrêté n°12258/2022 -MESupReS du 29 avril 2022  \n \nThe institution IFAGI  ET PARAMED  \nAndravoahangy Ambony is situate in Andravoahangy Ambony and offe rs', metadata={'page': 27.0, 'source': '/home/yvesloic/Documents/informatique/Défi/text-generative-chatbot-using-llama2/data/dataOfUniversity.pdf'}), Document(page_content='The institution ISFP Leader Andafiatsimo Tanjombato is situate in \nTanjombato and offers Sciences de la Santé/Licence w ith emphasis on -      \nSage-femme \n-      Infirmier and have government approbation referenceArrêté \nn°32794/2015 - MESupReS du 03 novembre 2015  \n \nThe institution ISFP Leader Ambovombe  Androy is situat

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [20]:
llm=CTransformers(model="/home/yvesloic/Documents/informatique/Défi/text-generative-chatbot-using-llama2/model/llama-2-7b-chat.ggmlv3.q4_0(1).bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_database_index.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [22]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"query": user_input})
    print("Response : ", result["result"])

Response :  ISPM stands for "Institution Supérieure des Professions de la Santé de Madagascar".
